In [195]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/emoji-classifer/train_emoji.csv
/kaggle/input/emoji-classifer/test_emoji.csv
/kaggle/input/glove-vector/glove.6B.50d.txt


In [196]:
!pip install emoji

In [197]:
import emoji

In [198]:
emoji.EMOJI_UNICODE

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [199]:
emoji_dictionary = {"0" : "\u2764\uFE0F",
                    "1" : ":baseball:",
                    "2" : ":grinning_face_with_big_eyes:",
                    "3" : ":disappointed_face:",
                    "4" : ":fork_and_knife:",
                    "5" : ":hundred_points:",
                    "6" : ":fire:",
                    "7" : ":face_blowing_a_kiss:",
                    "8" : ":chestnut:",
                    "9" : ":flexed_biceps:"
                   }

In [200]:
#emoji.emojize(":flexed_biceps:")

In [201]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


**Processing a custom dataset**

In [202]:
test= pd.read_csv("../input/emoji-classifer/test_emoji.csv",header= None)
train= pd.read_csv("../input/emoji-classifer/train_emoji.csv",header= None)

In [203]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [204]:
###Let's print the sentences with emoji
data= train.values
print(data.shape)

(132, 4)


In [205]:
X_train= train[0]
Y_train= train[1]

X_test= test[0]
Y_test= test[1]

In [206]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


**Converting Sentences into Embeddings**

In [207]:
f= open("../input/glove-vector/glove.6B.50d.txt",encoding="utf-8")

In [208]:
embeddings_index={}
cnt=0

for line in f:
    values=line.split()
    word= values[0]
    coefs= np.asarray(values[1:],dtype='float')
    embeddings_index[word]= coefs
f.close()

In [209]:
emb_dim= embeddings_index["eat"].shape[0]
print(emb_dim)

50


**Converting Sentences into vectors(Embedding Layer Output)**

In [210]:
def embedding_output(X):
    maxLen= 10
    embedding_out= np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix]= X[ix].split()
        
        for ij in range(len(X[ix])):
            #go to every word in current (ix) sentence
            try:
                embedding_out[ix][ij]= embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]= np.zeros((50,))
                
    return embedding_out
                
embeddings_matrix_train= embedding_output(X_train)
embeddings_matrix_test= embedding_output(X_test)    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [211]:
print(X_train[0])
print(len(X_train[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [212]:
print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [213]:
from keras.utils import to_categorical

In [214]:
Y_train= to_categorical(Y_train,num_classes= 5)
Y_test= to_categorical(Y_test,num_classes= 5)

print(Y_train.shape)
print(Y_train[0])

(132, 5)
[0. 0. 0. 1. 0.]


**Define RNN/LSTM Architecture**

In [215]:
from keras.models import Sequential
from keras.layers import *

In [216]:
model= Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences= True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences= False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_15 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_16 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________

In [217]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint= ModelCheckpoint("best_model.h5", monitor="val_loss", verbose=True, save_best_only=True)
earlystop= EarlyStopping(monitor='val_acc',patience=10)
hist= model.fit(embeddings_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True, validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 1s 13ms/step - loss: 1.5826 - acc: 0.2286 - val_loss: 1.5846 - val_acc: 0.2222
Epoch 2/100
105/105 [==============================] - 0s 530us/step - loss: 1.5533 - acc: 0.3238 - val_loss: 1.5888 - val_acc: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 477us/step - loss: 1.5277 - acc: 0.2857 - val_loss: 1.6022 - val_acc: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 467us/step - loss: 1.4912 - acc: 0.3429 - val_loss: 1.6235 - val_acc: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 460us/step - loss: 1.4841 - acc: 0.3524 - val_loss: 1.6335 - val_acc: 0.2963
Epoch 6/100
105/105 [==============================] - 0s 509us/step - loss: 1.4691 - acc: 0.3524 - val_loss: 1.6406 - val_acc: 0.2963
Epoch 7/100
105/105 [==============================] - 0s 454us/step - loss: 1.4204 - acc: 0.3810 - val_loss: 1.6405 - val_acc: 0.2593
Epoch 8/100

Epoch 61/100
105/105 [==============================] - 0s 495us/step - loss: 0.0393 - acc: 1.0000 - val_loss: 1.5247 - val_acc: 0.6296
Epoch 62/100
105/105 [==============================] - 0s 475us/step - loss: 0.0769 - acc: 0.9810 - val_loss: 1.8273 - val_acc: 0.5556
Epoch 63/100
105/105 [==============================] - 0s 482us/step - loss: 0.0634 - acc: 0.9905 - val_loss: 2.0025 - val_acc: 0.5556
Epoch 64/100
105/105 [==============================] - 0s 491us/step - loss: 0.1035 - acc: 0.9810 - val_loss: 1.7381 - val_acc: 0.5926
Epoch 65/100
105/105 [==============================] - 0s 482us/step - loss: 0.0288 - acc: 1.0000 - val_loss: 1.6573 - val_acc: 0.6296
Epoch 66/100
105/105 [==============================] - 0s 483us/step - loss: 0.0397 - acc: 0.9905 - val_loss: 1.7218 - val_acc: 0.5926
Epoch 67/100
105/105 [==============================] - 0s 496us/step - loss: 0.0449 - acc: 0.9905 - val_loss: 2.0010 - val_acc: 0.5926
Epoch 68/100
105/105 [==========================

In [218]:
pred= model.predict_classes(embeddings_matrix_test)
print(pred)

[4 3 3 0 2 2 1 2 4 2 1 2 0 3 1 3 3 2 3 4 0 0 4 2 3 1 2 0 0 2 0 1 0 2 0 2 2
 4 2 2 1 0 0 2 2 2 2 2 0 1 3 0 3 2 3 0]


In [219]:
model.evaluate(embeddings_matrix_test,Y_test)

56/56 [==============================] - 0s 237us/step


[2.1751607145581926, 0.5714285969734192]

In [228]:
pred= model.predict_classes(embeddings_matrix_test)

In [230]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))
    

never talk to me again
🍴
🍴
I am proud of your achievements
😞
😞
It is the worst day in my life
😃
😞
Miss you so much
❤️
❤️
food is life
😃
😃
I love you mum
❤️
😃
Stop saying bullshit
❤️
⚾
congratulations on your acceptance
❤️
😃
The assignment is too long
🍴
🍴
I want to go play
😃
😃
she did not answer my text
⚾
⚾
Your stupidity has no limit
😞
😃
how many points did he score
😞
❤️
my algorithm performs poorly
😃
😞
I got approved
⚾
⚾
Stop shouting at me
😞
😞
Sounds like a fun plan ha ha
😞
😞
no one likes him
😃
😃
the game just finished
😞
😞
I will celebrate soon
🍴
🍴
So sad you are not coming
❤️
❤️
She is my dearest love
😃
❤️
Good job
🍴
🍴
It was funny lol
😞
😃
candy is life
😞
😞
The chicago cubs won again
😞
⚾
I am hungry
😞
😃
I am so excited to see you after so long
❤️
❤️
you did well on you exam
😃
❤️
lets brunch some day
❤️
😃
